<center><h1> Edogeneity detection and instrument selection via solar-based graph learning (log) </h1></center>
<center><h2> Author: Ning Xu </h2></center>
<center><h3> xuningandy@outlook.com </h3></center>

Please note that
* this package is a purely CPU-based python package
* for the latest version of the package, click [here](https://github.com/isaac2math)

# #1. reset python kernel

In [1]:
%reset -f

---

# #2. import packages

In [2]:
import numpy             as np
import statsmodels.api   as sm
import pandas            as pd
import matplotlib.pyplot as plt
import statsmodels.api   as sm

from sklearn.linear_model       import LassoLarsCV, LassoCV, ElasticNetCV
from solar                      import solar
from statsmodels.iolib.summary2 import summary_col
from linearmodels.iv            import IV2SLS, IVGMM, IVGMMCUE  #IV regression and test
from collections                import OrderedDict
from linearmodels.iv.results    import compare

---

# #3. load house price data

In [3]:
linTB = pd.read_csv("Data2010.csv")

## #3(a). rename variables

In [4]:
linTB.rename(columns={'Tot_P_P':'TotPop',
                      'Mean_age_persons':'AgeSA1',
                      'Australian_citizen_P':'Aus',
                      'Average_num_psns_per_bedroom':'psn-per-bedroom',
                      'Average_household_size':'household_size',
                      'FinalResultEventPrice': 'Price',
                      'Lang_spoken_home_Eng_only_P':'Eng-speak',
                      'Mean_mortgage_repay_monthly':'Mortgage',
                      'Mean_Tot_prsnl_inc_weekly':'Inc',
                      'Mean_rent_weekly':'Rent',
                      'Mean_Tot_fam_inc_weekly':'FamInc'}, 
                 inplace=True)
linTB = linTB.drop(['Unnamed: 0'],axis=1)
linTB = linTB.dropna()
linTB['const'] = 1
linTB.columns

#drop larger than 1500
#linTB['index'] = linTB['AreaSize'] <=4000
#linTB = linTB[linTB['index']==1]

Index(['TotPop', 'Eng-speak', 'Aus', 'AgeSA1', 'Mortgage', 'Inc', 'Rent',
       'FamInc', 'psn-per-bedroom', 'household_size', 'TVO2010', 'TPO2010',
       'TVO2009', 'TPO2009', 'Suburb_Area', 'AreaSize', 'Bedrooms', 'Baths',
       'Parking', 'Airport', 'Beach', 'Cemetery', 'ChildCare',
       'CommunityFacility', 'Club', 'Gaol', 'GeneralHospital', 'GolfCourt',
       'High', 'Lib', 'MedCenter', 'Museum', 'Park', 'PO', 'Police',
       'PreSchool', 'PrimaryHigh', 'Primary', 'RailStat', 'Rubbish', 'Sewage',
       'SportsCenter', 'SportsCourtField', 'Swimming', 'Tertiary', 'DistBound',
       'ICSEA', 'ReadingY3', 'WritingY3', 'SpellingY3', 'GrammarY3',
       'NumeracyY3', 'ReadingY5', 'WritingY5', 'SpellingY5', 'GrammarY5',
       'NumeracyY5', 'Price', 'const'],
      dtype='object')

In [5]:
linTB.to_csv('House2010_linear.csv')
Y_linear = linTB[['Price']]
X_linear = linTB.drop(columns=['Price'])
X_linear.columns

Index(['TotPop', 'Eng-speak', 'Aus', 'AgeSA1', 'Mortgage', 'Inc', 'Rent',
       'FamInc', 'psn-per-bedroom', 'household_size', 'TVO2010', 'TPO2010',
       'TVO2009', 'TPO2009', 'Suburb_Area', 'AreaSize', 'Bedrooms', 'Baths',
       'Parking', 'Airport', 'Beach', 'Cemetery', 'ChildCare',
       'CommunityFacility', 'Club', 'Gaol', 'GeneralHospital', 'GolfCourt',
       'High', 'Lib', 'MedCenter', 'Museum', 'Park', 'PO', 'Police',
       'PreSchool', 'PrimaryHigh', 'Primary', 'RailStat', 'Rubbish', 'Sewage',
       'SportsCenter', 'SportsCourtField', 'Swimming', 'Tertiary', 'DistBound',
       'ICSEA', 'ReadingY3', 'WritingY3', 'SpellingY3', 'GrammarY3',
       'NumeracyY3', 'ReadingY5', 'WritingY5', 'SpellingY5', 'GrammarY5',
       'NumeracyY5', 'const'],
      dtype='object')

## #3(c). drop 0

### return the observations with zero in those variables

In [6]:
logTB = linTB
logTB = logTB[(logTB != 0).all(1)]

## #3(d). generate log variables

In [7]:
logTB[['logPrice']]     = np.log(logTB[['Price']])
logTB[['logMortgage']]  = np.log(logTB[['Mortgage']])
logTB[['logRent']]      = np.log(logTB[['Rent']])
logTB[['logFamInc']]    = np.log(logTB[['FamInc']])
logTB[['logPersonInc']] = np.log(logTB[['Inc']])

logTB = logTB.drop(['Price','Mortgage', 'Rent', 'FamInc', 'Inc'], axis=1)

/home/ning/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [8]:
logTB.to_csv('House2010_log.csv')
X_log = logTB.drop(columns=['logPrice'])
Y_log = logTB[['logPrice']]
X_log.columns

Index(['TotPop', 'Eng-speak', 'Aus', 'AgeSA1', 'psn-per-bedroom',
       'household_size', 'TVO2010', 'TPO2010', 'TVO2009', 'TPO2009',
       'Suburb_Area', 'AreaSize', 'Bedrooms', 'Baths', 'Parking', 'Airport',
       'Beach', 'Cemetery', 'ChildCare', 'CommunityFacility', 'Club', 'Gaol',
       'GeneralHospital', 'GolfCourt', 'High', 'Lib', 'MedCenter', 'Museum',
       'Park', 'PO', 'Police', 'PreSchool', 'PrimaryHigh', 'Primary',
       'RailStat', 'Rubbish', 'Sewage', 'SportsCenter', 'SportsCourtField',
       'Swimming', 'Tertiary', 'DistBound', 'ICSEA', 'ReadingY3', 'WritingY3',
       'SpellingY3', 'GrammarY3', 'NumeracyY3', 'ReadingY5', 'WritingY5',
       'SpellingY5', 'GrammarY5', 'NumeracyY5', 'const', 'logMortgage',
       'logRent', 'logFamInc', 'logPersonInc'],
      dtype='object')

---

# #4. generate X and Y for variable selection

In [9]:
X_linear_input = np.array(X_linear)
Y_linear_input = np.array(Y_linear)
X_log_input    = np.array(X_log)
Y_log_input    = np.array(Y_log)

In [10]:
obs     = X_linear_input.shape[0]
obs_log = X_log_input.shape[0]

Y_linear_input.shape = (obs,)
Y_log_input.shape    = (obs_log,)

---

# #5. compute CV coordinate descend (CV-cd), CV lars for lasso (CV-lars-lasso), CV elastic net (CV-en) and solar regression 

In [11]:
# CV coordinate descend (CV-cd)

RegLog1 = LassoCV(cv=10, normalize = True, random_state=0)
RegLog1.fit(X_log_input, Y_log_input)

LogLassoCV_ceof = RegLog1.coef_
LogLassoCV_ind  = np.nonzero(LogLassoCV_ceof)


# CV-lars-lasso

np.random.seed(0)

RegLog2 = LassoLarsCV(cv=10)
RegLog2.fit(X_log_input,Y_log_input)

LogLasso_lars_ceof = RegLog2.coef_
LogLasso_lars_ind  = np.nonzero(LogLasso_lars_ceof)

# CV-en

RegLog3 = ElasticNetCV(normalize=True, random_state=0)
RegLog3.fit(X_log_input,Y_log_input)

# solar

Y_log_input.shape = (obs_log,1)

np.random.seed(0)

RegLog4 = solar( X_log_input,Y_log_input, 10, -0.02, lasso = False)
LogSolar_coef, LogOpt_c, LogTest_error, LogQc_list, _, _, _, _ = RegLog4.fit()

LogSolar_ind = np.nonzero(LogSolar_coef)[0]

---

# #8. the results of variable selection

## #6(c). variable selection by lasso, solar and elastic net

### variables selected by CV-lars-lasso

In [12]:
LogLasso_lars_active = X_log.columns[LogLasso_lars_ind]
print(np.count_nonzero(LogLasso_lars_ind))
print(LogLasso_lars_active)

35
Index(['TotPop', 'AgeSA1', 'psn-per-bedroom', 'household_size', 'TPO2010',
       'Suburb_Area', 'AreaSize', 'Bedrooms', 'Baths', 'Parking', 'Airport',
       'Beach', 'ChildCare', 'Club', 'GeneralHospital', 'GolfCourt', 'High',
       'MedCenter', 'Museum', 'PO', 'Police', 'PreSchool', 'PrimaryHigh',
       'Primary', 'SportsCenter', 'SportsCourtField', 'Swimming', 'Tertiary',
       'DistBound', 'ICSEA', 'ReadingY3', 'WritingY3', 'NumeracyY3',
       'logMortgage', 'logRent', 'logFamInc'],
      dtype='object')


### variables selected by CV-cd

In [13]:
LogLassoCV_active = X_log.columns[LogLassoCV_ind]
print(np.count_nonzero(LogLassoCV_ceof))
print(LogLassoCV_active)

36
Index(['TotPop', 'AgeSA1', 'psn-per-bedroom', 'household_size', 'TPO2010',
       'Suburb_Area', 'AreaSize', 'Bedrooms', 'Baths', 'Parking', 'Airport',
       'Beach', 'ChildCare', 'Club', 'GeneralHospital', 'GolfCourt', 'High',
       'MedCenter', 'Museum', 'PO', 'Police', 'PreSchool', 'PrimaryHigh',
       'Primary', 'SportsCenter', 'SportsCourtField', 'Swimming', 'Tertiary',
       'DistBound', 'ICSEA', 'ReadingY3', 'WritingY3', 'NumeracyY3',
       'logMortgage', 'logRent', 'logFamInc'],
      dtype='object')


### variables selected by solar

In [14]:
LogSolar_active = X_log.columns[LogSolar_ind]
print(np.count_nonzero(LogSolar_coef))
print(LogSolar_active)

11
Index(['AgeSA1', 'Bedrooms', 'Baths', 'Parking', 'Beach', 'ChildCare', 'Gaol',
       'ICSEA', 'logMortgage', 'logRent', 'logFamInc'],
      dtype='object')


### variables selected by CV-en

In [15]:
LogEn_coef   = RegLog3.coef_
LogEn_ind    = np.nonzero(LogEn_coef)
LogEn_active = X_log.columns[LogEn_ind]
print(np.count_nonzero(LogEn_coef))
print(LogEn_active)

55
Index(['TotPop', 'Eng-speak', 'Aus', 'AgeSA1', 'psn-per-bedroom',
       'household_size', 'TVO2010', 'TPO2010', 'TVO2009', 'TPO2009',
       'Suburb_Area', 'AreaSize', 'Bedrooms', 'Baths', 'Parking', 'Airport',
       'Beach', 'ChildCare', 'CommunityFacility', 'Club', 'Gaol',
       'GeneralHospital', 'GolfCourt', 'High', 'MedCenter', 'Museum', 'Park',
       'PO', 'Police', 'PreSchool', 'PrimaryHigh', 'Primary', 'RailStat',
       'Rubbish', 'Sewage', 'SportsCenter', 'SportsCourtField', 'Swimming',
       'Tertiary', 'DistBound', 'ICSEA', 'ReadingY3', 'WritingY3',
       'SpellingY3', 'GrammarY3', 'NumeracyY3', 'ReadingY5', 'WritingY5',
       'SpellingY5', 'GrammarY5', 'NumeracyY5', 'logMortgage', 'logRent',
       'logFamInc', 'logPersonInc'],
      dtype='object')


# #9. Post-selection regression

## #7(a). post-elastic-net OLS

In [16]:
X_LogEn   = X_log[LogEn_active]
X_LogEn   = sm.add_constant(X_LogEn)
OLS_LogEn = sm.OLS(Y_log, X_LogEn)

OLS_LogEn_result = OLS_LogEn.fit()
print(OLS_LogEn_result.summary())

                            OLS Regression Results                            
Dep. Variable:               logPrice   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     699.8
Date:                Sat, 21 Dec 2019   Prob (F-statistic):               0.00
Time:                        21:09:07   Log-Likelihood:                -661.36
No. Observations:               11796   AIC:                             1435.
Df Residuals:                   11740   BIC:                             1848.
Df Model:                          55                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 8.7757      0.17

/home/ning/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


## #7(b). post-lasso (CV-cd) OLS

In [17]:
X_LogLassoCV   = X_log[LogLassoCV_active]
Y_LogLassoCV   = Y_log
X_LogLassoCV   = sm.add_constant(X_LogLassoCV)
OLS_LogLassoCV = sm.OLS(Y_LogLassoCV, X_LogLassoCV)

OLS_LogLassoCV_result = OLS_LogLassoCV.fit()
print(OLS_LogLassoCV_result.summary())

                            OLS Regression Results                            
Dep. Variable:               logPrice   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     1062.
Date:                Sat, 21 Dec 2019   Prob (F-statistic):               0.00
Time:                        21:09:07   Log-Likelihood:                -700.48
No. Observations:               11796   AIC:                             1475.
Df Residuals:                   11759   BIC:                             1748.
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                8.7614      0.151  

## #7(c). post-lasso (lars-lasso) OLS

In [18]:
X_LogLars   = X_log[LogLasso_lars_active]
Y_LogLars   = Y_log
X_LogLars   = sm.add_constant(X_LogLars)
OLS_LogLars = sm.OLS(Y_LogLars, X_LogLars)

OLS_LogLars_result = OLS_LogLars.fit()
print(OLS_LogLars_result.summary())

                            OLS Regression Results                            
Dep. Variable:               logPrice   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     1062.
Date:                Sat, 21 Dec 2019   Prob (F-statistic):               0.00
Time:                        21:09:07   Log-Likelihood:                -700.48
No. Observations:               11796   AIC:                             1475.
Df Residuals:                   11759   BIC:                             1748.
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                8.7614      0.151  

## #7(d). post-solar OLS

In [19]:
X_Gaol     = X_log[LogSolar_active]
X_Gaol     = sm.add_constant(X_Gaol)
OLS_LogSolar  = sm.OLS(Y_log, X_Gaol)
OLS_LogSolar_result = OLS_LogSolar.fit()
print(OLS_LogSolar_result.summary())

                            OLS Regression Results                            
Dep. Variable:               logPrice   R-squared:                       0.730
Model:                            OLS   Adj. R-squared:                  0.730
Method:                 Least Squares   F-statistic:                     2900.
Date:                Sat, 21 Dec 2019   Prob (F-statistic):               0.00
Time:                        21:09:07   Log-Likelihood:                -1507.3
No. Observations:               11796   AIC:                             3039.
Df Residuals:                   11784   BIC:                             3127.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           7.2109      0.105     68.413      

## #7(e). generate a comparison table for post-selection OLS result (slope, significance, $R^2$)

In [20]:
info_dict={'R-squared' : lambda x: f"{x.rsquared:.2f}",
           'No. observations' : lambda x: f"{int(x.nobs):d}"}

results_table = summary_col(results=[OLS_LogEn_result, OLS_LogLassoCV_result, 
                                     OLS_LogLars_result, OLS_LogSolar_result],
                            float_format='%0.2f',
                            stars = True,
                            model_names=['post-en',
                                         'post-lasso(lars)',
                                         'post-larsso(cd)',
                                         'post-solar'],
                            info_dict=info_dict,
                            regressor_order=['const',
                                             'age',
                                             'Bedrooms',
                                             'Baths',
                                             'Parking',
                                             'Beach',
                                             'ChildCare',
                                             'Gaol',
                                             'ICSEA',
                                             'logMortage',
                                             'logRent',
                                             'logFamInc'])

results_table.add_title('Table 1 - Post-selection OLS comparison')

print(results_table)

               Table 1 - Post-selection OLS comparison
                  post-en  post-lasso(lars) post-larsso(cd) post-solar
----------------------------------------------------------------------
const             8.78***  8.76***          8.76***         7.21***   
                  (0.17)   (0.15)           (0.15)          (0.11)    
Bedrooms          0.21***  0.21***          0.21***         0.23***   
                  (0.00)   (0.00)           (0.00)          (0.00)    
Baths             0.09***  0.10***          0.10***         0.09***   
                  (0.00)   (0.00)           (0.00)          (0.00)    
Parking           0.08***  0.08***          0.08***         0.08***   
                  (0.00)   (0.00)           (0.00)          (0.00)    
Beach             -2.22*** -2.21***         -2.21***        -2.45***  
                  (0.33)   (0.11)           (0.11)          (0.14)    
ChildCare         -4.27*** -4.49***         -4.49***        -2.45***  
                  (0.2

# #8. rectified solar selection

## #8(a). generate correlation table of all variables selected by solar

In [21]:
X_log[LogSolar_active].corr()

,AgeSA1,Bedrooms,Baths,Parking,Beach,ChildCare,Gaol,ICSEA,logMortgage,logRent,logFamInc
AgeSA1,1.000000,0.310088,0.166457,0.217984,0.012365,0.180460,0.124008,0.017711,0.172088,-0.057359,0.120974
Bedrooms,0.310088,1.000000,0.639381,0.477543,0.032923,0.243481,0.180793,-0.043061,0.173557,0.023377,0.094320
Baths,0.166457,0.639381,1.000000,0.427226,-0.075538,0.051809,0.026854,0.098237,0.215537,0.156228,0.199941
Parking,0.217984,0.477543,0.427226,1.000000,-0.008247,0.149590,0.091177,-0.022136,0.128067,0.043028,0.091189
Beach,0.012365,0.032923,-0.075538,-0.008247,1.000000,0.041460,0.532753,-0.259973,-0.231921,-0.251981,-0.340208
ChildCare,0.180460,0.243481,0.051809,0.149590,0.041460,1.000000,0.750484,-0.396499,-0.210664,-0.227383,-0.236176
Gaol,0.124008,0.180793,0.026854,0.091177,0.532753,0.750484,1.000000,-0.234434,-0.187952,-0.196900,-0.173732
ICSEA,0.017711,-0.043061,0.098237,-0.022136,-0.259973,-0.396499,-0.234434,1.000000,0.346822,0.364504,0.568851
logMortgage,0.172088,0.173557,0.215537,0.128067,-0.231921,-0.210664,-0.187952,0.346822,1.000000,0.306933,0.551085
logRent,-0.057359,0.023377,0.156228,0.043028,-0.251981,-0.227383,-0.196900,0.364504,0.306933,1.000000,0.482644


## #8(b). compute $\mathrm{corr} \left( \cdot , \mathrm{goal} \right)$ of all variables

In [22]:
goal_corr_table = X_log.corr()['Gaol']

### focus on those with $\left\vert \mathrm{corr} \left( \cdot , \mathrm{goal} \right) \right\vert \geqslant 0.3$

In [23]:
goal_corr_table[np.abs(goal_corr_table)>=0.3]

household_size       0.304713
Airport              0.708315
Beach                0.532753
Cemetery             0.404004
ChildCare            0.750484
CommunityFacility    0.317501
Club                 0.397920
Gaol                 1.000000
GeneralHospital      0.464019
GolfCourt            0.468939
PreSchool            0.468674
Rubbish              0.662911
Sewage               0.453901
Swimming             0.375316
Tertiary             0.481035
Name: Gaol, dtype: float64

### focus on those with $\left\vert \mathrm{corr} \left( \cdot , \mathrm{goal} \right) \right\vert \geqslant 0.5$

In [24]:
goal_corr_table[np.abs(goal_corr_table)>=0.5]

Airport      0.708315
Beach        0.532753
ChildCare    0.750484
Gaol         1.000000
Rubbish      0.662911
Name: Gaol, dtype: float64

## #8(c). check irrepresentable condition 

## $\mathrm{Gaol} = X_{Goal}\gamma + e$, where $X_{Goal} = \left[ \mathrm{Airport}, \mathrm{ChildCare}, \mathrm{Rubbish}, \mathrm{Beach} \right]$

In [25]:
X_Gaol_group = X_log[['Airport', 'ChildCare', 'Rubbish', 'Beach']]
Y_Gaol_group = X_log[['Gaol']]

Y_Gaol_group = (Y_Gaol_group - Y_Gaol_group.mean()) / Y_Gaol_group.std()
X_Gaol_group = (X_Gaol_group - X_Gaol_group.mean()) / X_Gaol_group.std()

X_Gaol_group = sm.add_constant(X_Gaol_group)

OLS_Gaol = sm.OLS(Y_Gaol_group, X_Gaol_group)

OLS_Gaol_result = OLS_Gaol.fit()
print(OLS_Gaol_result.summary())

                            OLS Regression Results                            
Dep. Variable:                   Gaol   R-squared:                       0.883
Model:                            OLS   Adj. R-squared:                  0.883
Method:                 Least Squares   F-statistic:                 2.230e+04
Date:                Sat, 21 Dec 2019   Prob (F-statistic):               0.00
Time:                        21:09:08   Log-Likelihood:                -4069.7
No. Observations:               11796   AIC:                             8149.
Df Residuals:                   11791   BIC:                             8186.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -8.965e-15      0.003  -2.85e-12      1.0

### check if $\left\Vert \gamma \right\Vert_1 \geqslant 1$

In [26]:
print('the abs value of regression coef: ', np.sum(np.abs(OLS_Gaol_result.params)))

the abs value of regression coef:  1.3683947330011708


## #8(c). due to the potential IRC violation, add 'Airport' and 'Rubbish' back to post-solar OLS (referred to as the rectified post-solar)

In [27]:
X_rec = X_log[['logMortgage', 'logFamInc', 'logRent', 'AgeSA1',
                 'Bedrooms', 'Baths', 'Parking', 'Beach', 'ICSEA', 'Gaol', 
                 'Airport', 'ChildCare', 'Rubbish']]

X_rec   = sm.add_constant(X_rec)
OLS_rec = sm.OLS(Y_log, X_rec)

OLS_rec_result = OLS_rec.fit()
print(OLS_rec_result.summary())

                            OLS Regression Results                            
Dep. Variable:               logPrice   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.740
Method:                 Least Squares   F-statistic:                     2587.
Date:                Sat, 21 Dec 2019   Prob (F-statistic):               0.00
Time:                        21:09:08   Log-Likelihood:                -1276.2
No. Observations:               11796   AIC:                             2580.
Df Residuals:                   11782   BIC:                             2684.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           7.9898      0.111     71.785      

In [28]:
info_dict={'R-squared' : lambda x: f"{x.rsquared:.2f}",
           'No. observations' : lambda x: f"{int(x.nobs):d}"}

results_table2 = summary_col(results=[OLS_LogEn_result, OLS_LogLassoCV_result, 
                                      OLS_LogLars_result, OLS_rec_result, 
                                      OLS_LogSolar_result],
                            float_format='%0.2f',
                            stars = True,
                            model_names=['post-en',
                                         'post-lasso(lars)',
                                         'post-larsso(cd)',
                                         'rec post-solar',
                                         'post-solar'],
                            info_dict=info_dict,
                            regressor_order=['const',
                                             'age',
                                             'Bedrooms',
                                             'Baths',
                                             'Parking',
                                             'Beach',
                                             'ChildCare',
                                             'Gaol',
                                             'ICSEA',
                                             'logMortgage',
                                             'logRent',
                                             'logFamInc',
                                             'Airport',
                                             'Rubbish'])

results_table2.add_title('Table 2 - Post-selection OLS comparison')

print(results_table2)

                       Table 2 - Post-selection OLS comparison
                  post-en  post-lasso(lars) post-larsso(cd) rec post-solar post-solar
-------------------------------------------------------------------------------------
const             8.78***  8.76***          8.76***         7.99***        7.21***   
                  (0.17)   (0.15)           (0.15)          (0.11)         (0.11)    
Bedrooms          0.21***  0.21***          0.21***         0.23***        0.23***   
                  (0.00)   (0.00)           (0.00)          (0.00)         (0.00)    
Baths             0.09***  0.10***          0.10***         0.09***        0.09***   
                  (0.00)   (0.00)           (0.00)          (0.00)         (0.00)    
Parking           0.08***  0.08***          0.08***         0.08***        0.08***   
                  (0.00)   (0.00)           (0.00)          (0.00)         (0.00)    
Beach             -2.22*** -2.21***         -2.21***        -1.34***       -2

In [29]:
X_corr = X_log[['logMortgage', 'logFamInc', 'logRent', 'AgeSA1',
                'Bedrooms', 'Baths', 'Parking', 'Beach', 'ICSEA', 'Gaol', 
                'Airport', 'ChildCare', 'Rubbish']]

Y_corr = (Y_log  - Y_log.mean() ) / Y_log.std()
X_corr = (X_corr - X_corr.mean()) / X_corr.std()

X_corr = sm.add_constant(X_corr)
OLS_corr = sm.OLS(Y_corr, X_corr)

OLS_corr_result = OLS_corr.fit()
print(OLS_corr_result.summary())

                            OLS Regression Results                            
Dep. Variable:               logPrice   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.740
Method:                 Least Squares   F-statistic:                     2587.
Date:                Sat, 21 Dec 2019   Prob (F-statistic):               0.00
Time:                        21:09:08   Log-Likelihood:                -8778.8
No. Observations:               11796   AIC:                         1.759e+04
Df Residuals:                   11782   BIC:                         1.769e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        1.651e-13      0.005   3.52e-11      

# #9. linear PGM estimation based on solar

## this part is finished using R package 'bnlearn', which is fully tested in debian and ubuntu (not tested in windows or mac). Please make sure that either your have R kernel in jupyter or you create R session in Python kernel.

# #10. endogeneity detection and IV estimation using linear PGM

## #10(a). Robust OLS that pretends nothing happens

In [30]:
OLS = IV2SLS(dependent=logTB['logRent'],
            exog=logTB[['const', 'TotPop', 'household_size', 
                        'Beach', 'ChildCare', 'Gaol', 'PrimaryHigh',
                        'ICSEA', 'logPersonInc','logFamInc', 'logPrice']],
            endog=None,
            instruments=None).fit(cov_type='robust')

print(OLS.summary)

                            OLS Estimation Summary                            
Dep. Variable:                logRent   R-squared:                      0.3650
Estimator:                        OLS   Adj. R-squared:                 0.3645
No. Observations:               11796   F-statistic:                    5437.3
Date:                Sat, Dec 21 2019   P-value (F-stat)                0.0000
Time:                        21:09:08   Distribution:                 chi2(10)
Cov. Estimator:                robust                                         
                                                                              
                               Parameter Estimates                                
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
const              1.8639     0.1075     17.337     0.0000      1.6532      2.0746
TotPop             0.0002  1.428e-05

In [31]:
OLS = IV2SLS(dependent=logTB['logRent'],
            exog=logTB[['const', 'TotPop', 'household_size', 
                        'Beach', 'ChildCare', 'Gaol', 'PrimaryHigh',
                        'ICSEA', 'logPersonInc','logFamInc', 'logPrice']],
            endog=None,
            instruments=None).fit(cov_type='robust')

print(OLS.summary)

                            OLS Estimation Summary                            
Dep. Variable:                logRent   R-squared:                      0.3650
Estimator:                        OLS   Adj. R-squared:                 0.3645
No. Observations:               11796   F-statistic:                    5437.3
Date:                Sat, Dec 21 2019   P-value (F-stat)                0.0000
Time:                        21:09:08   Distribution:                 chi2(10)
Cov. Estimator:                robust                                         
                                                                              
                               Parameter Estimates                                
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
const              1.8639     0.1075     17.337     0.0000      1.6532      2.0746
TotPop             0.0002  1.428e-05

## #10(b). Robust 2SLS

In [32]:
iv = IV2SLS(dependent=logTB['logRent'],
            exog=logTB[['const', 'TotPop', 'household_size', 
                        'Beach', 'ChildCare', 'Gaol', 'PrimaryHigh',
                        'ICSEA','logPersonInc','logFamInc']],
            endog=logTB[['logPrice']],
            instruments=logTB[['Baths']]).fit(cov_type='robust')

print(iv.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                logRent   R-squared:                      0.3632
Estimator:                    IV-2SLS   Adj. R-squared:                 0.3627
No. Observations:               11796   F-statistic:                    5355.1
Date:                Sat, Dec 21 2019   P-value (F-stat)                0.0000
Time:                        21:09:08   Distribution:                 chi2(10)
Cov. Estimator:                robust                                         
                                                                              
                               Parameter Estimates                                
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
const              1.6026     0.1444     11.101     0.0000      1.3197      1.8856
TotPop             0.0002  1.459e-05

In [33]:
iv2 = IV2SLS(dependent=logTB['logRent'],
            exog=logTB[['const', 'TotPop', 'household_size', 
                        'Beach', 'ChildCare', 'Gaol', 'PrimaryHigh',
                        'ICSEA','logPersonInc','logFamInc']],
            endog=logTB[['logPrice']],
            instruments=logTB[['Bedrooms']]).fit(cov_type='robust')

print(iv2.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                logRent   R-squared:                      0.3618
Estimator:                    IV-2SLS   Adj. R-squared:                 0.3612
No. Observations:               11796   F-statistic:                    5485.6
Date:                Sat, Dec 21 2019   P-value (F-stat)                0.0000
Time:                        21:09:09   Distribution:                 chi2(10)
Cov. Estimator:                robust                                         
                                                                              
                               Parameter Estimates                                
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
const              2.2169     0.1237     17.917     0.0000      1.9744      2.4594
TotPop             0.0002  1.449e-05

In [34]:
iv3 = IV2SLS(dependent=logTB['logRent'],
            exog=logTB[['const', 'TotPop', 'household_size', 
                        'Beach', 'ChildCare', 'Gaol', 'PrimaryHigh',
                        'ICSEA','logPersonInc','logFamInc']],
            endog=logTB[['logPrice']],
            instruments=logTB[['Parking']]).fit(cov_type='robust')

print(iv3.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                logRent   R-squared:                      0.3649
Estimator:                    IV-2SLS   Adj. R-squared:                 0.3644
No. Observations:               11796   F-statistic:                    5457.8
Date:                Sat, Dec 21 2019   P-value (F-stat)                0.0000
Time:                        21:09:09   Distribution:                 chi2(10)
Cov. Estimator:                robust                                         
                                                                              
                               Parameter Estimates                                
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
const              1.9133     0.1625     11.777     0.0000      1.5949      2.2318
TotPop             0.0002  1.477e-05

## #10(c). Endogeneity test
### Durbin Endogeneity test

In [35]:
iv.durbin()

Durbin test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 13.0337
P-value: 0.0003
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f2a0e654350

In [36]:
iv2.durbin()

Durbin test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 49.5049
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f2a04621bd0

In [37]:
iv3.durbin()

Durbin test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 0.2538
P-value: 0.6144
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f2a047bb610

### Hausman Endogeneity test

In [38]:
iv.wu_hausman()

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 13.0348
P-value: 0.0003
Distributed: F(1,11784)
WaldTestStatistic, id: 0x7f2a047bb790

In [39]:
iv2.wu_hausman()

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 49.6629
P-value: 0.0000
Distributed: F(1,11784)
WaldTestStatistic, id: 0x7f2a044bffd0

In [40]:
iv3.wu_hausman()

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 0.2535
P-value: 0.6146
Distributed: F(1,11784)
WaldTestStatistic, id: 0x7f2a04618b10

### Wooldridge regression-based endogeneity test

In [41]:
iv.wooldridge_regression

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 9.1282
P-value: 0.0025
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f2a044aa610

In [42]:
iv2.wooldridge_regression

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 31.6738
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f2a0c45f110

In [43]:
iv3.wooldridge_regression

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 0.1725
P-value: 0.6779
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f2a044aab10

### Wooldridge score endogeneity test

In [44]:
iv.wooldridge_score

Wooldridge's score test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 9.2345
P-value: 0.0024
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f2a04414310

In [45]:
iv2.wooldridge_score

Wooldridge's score test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 33.7213
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f2a04455210

In [46]:
iv3.wooldridge_score

Wooldridge's score test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 0.1734
P-value: 0.6771
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f2a04621310

In [47]:
IV_result = OrderedDict()
IV_result['OLS'] = OLS
IV_result['robust 2SLS'] = iv
IV_result['weak 2SLS 1'] = iv2
IV_result['weak 2SLS 2'] = iv3
print(compare(IV_result))

                              Model Comparison                              
                               OLS   robust 2SLS   weak 2SLS 1   weak 2SLS 2
----------------------------------------------------------------------------
Dep. Variable              logRent       logRent       logRent       logRent
Estimator                      OLS       IV-2SLS       IV-2SLS       IV-2SLS
No. Observations             11796         11796         11796         11796
Cov. Est.                   robust        robust        robust        robust
R-squared                   0.3650        0.3632        0.3618        0.3649
Adj. R-squared              0.3645        0.3627        0.3612        0.3644
F-statistic                 5437.3        5355.1        5485.6        5457.8
P-value (F-stat)            0.0000        0.0000        0.0000        0.0000
==================     ===========   ===========   ===========   ===========
const                       1.8639        1.6026        2.2169        1.9133

# logRent variable selection : find the MB of logRent in our dataset

In [48]:
logRent = logTB['logRent']
X_logRent = logTB.drop(['logRent'], axis = 1)

Rent_input  = np.array(X_logRent)
Rent_output = np.array(logRent)

Rent_output.shape = (11796,1)

In [49]:
np.random.seed(0)

RegRent4 = solar(Rent_input, Rent_output, 10, -0.02, lasso = False)
RentSolar_coef, RentOpt_c, RentTest_error, RentQc_list, _, _, _, _ = RegRent4.fit()

RentSolar_ind = np.nonzero(RentSolar_coef)[0]

In [50]:
RentSolar_ind = X_logRent.columns[RentSolar_ind]
print(np.count_nonzero(RentSolar_ind))
print(RentSolar_ind)

10
Index(['TotPop', 'household_size', 'Beach', 'ChildCare', 'Gaol', 'PrimaryHigh',
       'ICSEA', 'logPrice', 'logFamInc', 'logPersonInc'],
      dtype='object')
